<a href="https://colab.research.google.com/github/hyeonji0401/CodingTest/blob/main/cudaMalloc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.1 CUDA 프로그램의 구조 및 흐름
호스트 코드가 반드시 필요한 이유

> CPU가 운영체제와 같은 컴퓨터 시스템의 기본 연산 장치이며, GPU와 같은 다른 연산 장치를 사용하기 위해서는 호스트 코드에서 커널을 호출해야 하기 때문임

* CPU와 GPU는 서로 독립된 장치로 사용하는 메모리 영역이 다름
* GPU는 디바이스 메모리를 사용하지만 모든 데이터는 기본적으로 호스트 메모리에 저장되어있음

=> GPU를 이용해서 데이터를 처리하기 위해서는 호스트 메모리에 있는 데이터를 디바이스 메모리로 복사해주어야함

<CUDA 프로그램 흐름>
1. 호스트 -> 디바이스 데이터 복사(GPU에서 처리할 데이터)
2. GPU연산
3. 디바이스 -> 호스트 데이터 복사(연산 결과 데이터)





# 3.2 CUDA 기초 메모리 API

**1. 디바이스 메모리 공간 할당 및 초기화**
> 디바이스 메모리로 데이터를 복사하기 위해서는 메모리에 사용할 공간을 할당받아야함

> 이는 C언어 malloc()함수와 동일함

> 차이점은 C언어는 호스트 공간의 메모리를 할당하는 것이고 cudaMalloc()은 디바이스 메모리 공간을 할당받는 것임

**디바이스 메모리 할당**
* cudaMalloc()

함수 원형


```
cudaError_t cudaMalloc(void **ptr, size_t size)
```
1. void **ptr
- 디바이스 메모리 공간의 시작 주소를 담을 포인터 변수의 주소

2. size_t size
- 할당할 공간의 크기(byte 단위)

3. 반환형
- cudaError_t 열거형




In [ ]:
%%cuda

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

int main(void)
{
    int *dDataPtr;
    cudaMalloc(&dDataPtr, sizeof(int)*32);
}


위의 코드는 디바이스 메모리 공간에 int형 데이터 32개를 담을 공간을 할당하는 예시임

위에서 설명했던 것과 같이 &dDatePtr가 할당된 메모리 공간의 시작주소가 저장되는 포인터 변수가 되는 것

* 주의할 점 : dDatePtr이 가리키는 주소는 디바이스 메모리 주소이므로 호스트 코드에서 직접 접근할 수 없다

그래서 만약


In [ ]:
%%cuda

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>

int main(void)
{
    int *dDataPtr;
    cudaMalloc(&dDataPtr, sizeof(int)*32);
    printf("%d", dDataPtr[0]);
    cudaFree(dDataPtr);
    return 0;
}


이와 같이 데이터에 접근하면 실행 오류가 발생함

**printf()는 CPU에서 작동하는 코드이기 때문임**



> CUDA 프로그램에서는 호스트 메모리 영역을 사용한느 변수와 디바이스 메모리 영역을 사용하는 변수를 구분하기 위해 일반적으로 디바이스 메모리 영역을 사용하는 변수의 이름 앞에 디바이스를 뜻하는 d를 붙여줌

* cudaFree()는 할당된 메모리를 해제하는 함수

**디바이스 메모리 해제**

* cudaFree()

함수 원형


```
cudaError_t cudaFree(void* ptr)
```
1.void *ptr
- 해제할 메모리 공간을 가리키는 포인터 변수

2. 반환형
- cudaError_t 열거형

**디바이스 메모리 초기화**

- cudaMemset()


> cudaMalloc()을 통해 메모리 공간을 할당받으면 해당 메모리 공간에 남아있던 쓰레기 값이 그대로 남아있으므로 초기화 해줘야함

함수 원형


```
cudaError_t cudaMemset(void *ptr, int value, size_t size)
```

1. void * ptr
- 값을 초기화할 메모리 공간의 시작 주소

2. int value
- 각 바이트를 초기화할 값

3. size_t size
- 초기화할 메모리 공간의 크기


**에러 코드 확인**

* cudaGetErrorName()


> CUDA API의 반환값 대부분은 에러코드(cudaError_t 열거형)이며 에러코드의 수는 그만큼 많음

> 각 에러코드 번호는 버전에 따라 변경될 수 있으며 모든 에러코드를 알 수는 없기 때문에 함수를 통해 에러의 종류를 확인하면 좋음

함수 원형


```
__host__ __device__ const char* cudaGetErrorName(cudaError_t error)
```

원형 앞에 __host__ 와 __device__ 가 모두 붙어 있는데, 이는 호스트와 디바이스 코드 모두에서 사용가능함을 말함


**디바이스 메모리 할당/초기화/해제 예제**

In [ ]:
%%cuda

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>

void checkDeviceMemory(void)
{
    size_t free, total;
    cudaMemGetInfo(&free, &total);
    printf("Device memory (free/total) = %lld/%lld bytes\n", free, total);
}

int main(void)
{
    int* dDataPtr;
    cudaError_t errorCode;

    checkDeviceMemory();
    errorCode = cudaMalloc(&dDataPtr, sizeof(int)*1024*1024);
    printf("cudaMalloc - %s\n", cudaGetErrorName(errorCode)); //메모리 할당 전
    checkDeviceMemory();

    errorCode = cudaMemset(dDataPtr, 0, sizeof(int)*1024*1024); //메모리 할당 후 가용 메모리 크기가 4MB 감소됨
    printf("cudaMalloc - %s\n", cudaGetErrorName(errorCode));

    errorCode = cudaFree(dDataPtr);
    printf("cudaFree - %s\n", cudaGetErrorName(errorCode)); //Free선언 후 다시 메모리 공간이 반환됨을 확인 할 수 있음
    checkDeviceMemory();
}

Device memory (free/total) = 15727656960/15835660288 bytes
cudaMalloc - cudaSuccess
Device memory (free/total) = 15723462656/15835660288 bytes
cudaMalloc - cudaSuccess
cudaFree - cudaSuccess
Device memory (free/total) = 15727656960/15835660288 bytes



cudaMalloc을 사용해서 int형 정수를 담을 수 있는 총 4MB의 공간을 할당하며, cudaMemset으로 모두 0으로 초기화하고 cudaFree로 메모리 공간을 해지하는 코드임


**[checkDeviceMemory()]**

디바이스 메모리의 총 크기와 현재 사용 가능한 공간의 크기를 반환해주는 함수로 cudaMemGetInfo()를 사용함

free: 현재 가용 메모리 크기

total: 사용하는 GPU가 가진 총 디바이스 메모리 크기

> 사용하는 GPU보다 큰 메모리를 할당할 경우 에러 발생함

**2. 호스트 - 디바이스 메모리 데이터 복사**



> 호스트 메모리와 디바이스 메모리는 서로 독립된 공간으로 다른 장치에 있는 데이터가 필요하다면 명시적인 데이터 복사 과정이 필요함

**장치 간 데이터 복사**

- cudaMemcpy()


> CUDA 프로그램에서 장치 간 데이터 복사를 위해 사용하는 함수

함수 원형


```
cudaError_t cudaMemcpy(void* dst, const void* src, size_t size, enum cudaMemcpyKind kind)
```
1. void* dist
- 복사될 메모리 공간의 시작 주소를 담고 있는 포인터 변수

2. const void* src
- 복사할 원본 데이터가 들어있는 메모리 공간의 시작 주소를 담고 있는 포인터 변수

3. size_t size
- 복사할 데이터의 크기(바이트 단위)

4. enum cudaMemcpyKind kind
- cudaMemcpyKind의 열거형 변수로 데이터 복사의 방향을 설정하는 인자
- 000ToXXX 형태로 000에서 XXX로 복사됨을 의미함
- 이 방향은 함수의 첫번째, 두번째 인자의 방향과 일치해야함

> cudaMemcpyDefault : dst와 src의 포인터 값에 의해 결정됨
- 이 경우 CUDA런타임이 방향성을 판단해서 복사를 수행해줌
- unified virtual addressing(호스트메모리와 디바이스 메모리를 하나의 공간처럼 가상화해줌)을 지원하는 시스템에서만 사용 가능
- 이 방법보다는 명시적으로 지정하는 것이 더 권장됨



